In [1]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.float16 if torch.cuda.is_available() else torch.float32
print(device)

cuda


In [2]:
from QwenWithPerceiverCrossAttn import QwenWithPerceiverCrossAttn
model = QwenWithPerceiverCrossAttn().to(device).to(dtype)

c:\Users\Charlie\anaconda3\envs\ai_assistant\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
W1225 18:41:58.775000 26716 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


Perceiver latent dimension: 512
Perceiver number of latents: 784
Created input projection: 2560 -> 704
Gradient checkpointing enabled
Cross-attention inserted at layer 8


In [3]:
from transformers import AutoTokenizer

qwen_model_name = "Bossologist/Qwen3-4B-Instruct-2507_general_ft_merged"
tokenizer_path = qwen_model_name
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path, trust_remote_code=True)

In [4]:
input = "Are you doing something later"
encoded = tokenizer(
    input,
    max_length=2048,
    truncation=True,
    padding=True,
    return_tensors="pt",
)
# input_ids must be Long (int64), not float16
input_ids = encoded["input_ids"].to(device).long()
attention_mask = encoded["attention_mask"].to(device)
input_ids[0]

tensor([11526,   498,  3730,  2494,  2937], device='cuda:0')

In [5]:
output = model(input_ids=input_ids, attention_mask=attention_mask)
# Convert logits to predicted token IDs and decode to text
logits = output[0][:, -1, :]
pred_ids = torch.argmax(logits, dim=-1)
pred_text = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
print(pred_text)

['2']


In [ ]:
num_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Number of trainable parameters: {num_trainable_params:,}")
sum([p.numel() for p in model.qwen_model.model.model.parameters() if p.requires_grad])


Number of trainable parameters: 77,642,369


2567191041